# Model Training with PyTorch

Train a model that predicts whether or not a patient has diabetes, based on medical features. 

### 1. Import the required libraries and packages.

In [2]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 128.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 177.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 188.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 502.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 152.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 145.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 180.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 184.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 173.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 176.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from typing import List, Dict

import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### 2. Load the data into a Pandas dataframe.

In [4]:
data = pd.read_csv('./data/diabetes.csv')

Split the data into two data frames: features (`X`) and target variable (`y`).

In [5]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

Inspect the two dataframes.

In [6]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [7]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

Divide the data into training and test data sets. 

The `train_test_split` method of Scikit-learn can split the data set into random train and test subsets.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=0
)

print(f"Number of samples in training set: {X_train.shape[0]}")
print(f"Number of samples in test set: {X_test.shape[0]}")

Number of samples in training set: 614
Number of samples in test set: 154


Encode the data as PyTorch tensors.

In [9]:
X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)

Preview the training features tensor and its shape.

In [10]:
X_train

tensor([[7.0000e+00, 1.5000e+02, 7.8000e+01,  ..., 3.5200e+01, 6.9200e-01,
         5.4000e+01],
        [4.0000e+00, 9.7000e+01, 6.0000e+01,  ..., 2.8200e+01, 4.4300e-01,
         2.2000e+01],
        [0.0000e+00, 1.6500e+02, 9.0000e+01,  ..., 5.2300e+01, 4.2700e-01,
         2.3000e+01],
        ...,
        [4.0000e+00, 9.4000e+01, 6.5000e+01,  ..., 2.4700e+01, 1.4800e-01,
         2.1000e+01],
        [1.1000e+01, 8.5000e+01, 7.4000e+01,  ..., 3.0100e+01, 3.0000e-01,
         3.5000e+01],
        [5.0000e+00, 1.3600e+02, 8.2000e+01,  ..., 0.0000e+00, 6.4000e-01,
         6.9000e+01]])

In [11]:
X_train.shape

torch.Size([614, 8])

Preview the training target value tensor and its shape.

In [12]:
y_train

tensor([1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
        1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [13]:
y_train.shape

torch.Size([614])

### 4. Create and train the model.

Define a simple neural network model with PyTorch.
The network must take eight input features and output two target values, corresponding to the two possible outcomes, diabetes or no diabetes.
The network also defines two internal layers, with 20 and 10 neurons respectively.

In [14]:
# Seed for reproducible results
torch.manual_seed(20)


class ANN_model(nn.Module):
    def __init__(
        self,
        num_input_features=8,
        num_neurons_layer1=20,
        num_neurons_layer2=10,
        num_targets=2
    ):
        super().__init__()
        # Define the neural network layers
        self.layer1 = nn.Linear(num_input_features, num_neurons_layer1)
        self.layer2 = nn.Linear(num_neurons_layer1, num_neurons_layer2)
        self.out = nn.Linear(num_neurons_layer2, num_targets)

    def forward(self, X):
        # pass the data through the layers
        x = F.relu(self.layer1(X))
        x = F.relu(self.layer2(x))
        return self.out(x)

Instantiate the model and define the loss function, the optimizer, and the training epochs.

In [15]:
model = ANN_model()

# == Backward Propagation Configuration ==
# CrossEntropyLoss is a common loss function for classifcation
loss_function = nn.CrossEntropyLoss()
# Use the Adam optimizer with a learning rate of 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 500

Train the model.

In [16]:
for i in range(epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(f"Epoch: {i}. Loss: {loss.item()}")

Epoch: 0. Loss: 7.187566757202148
Epoch: 10. Loss: 0.8532984256744385
Epoch: 20. Loss: 0.716244637966156
Epoch: 30. Loss: 0.6571372747421265
Epoch: 40. Loss: 0.6171259880065918
Epoch: 50. Loss: 0.6081922054290771
Epoch: 60. Loss: 0.6015171408653259
Epoch: 70. Loss: 0.5953478813171387
Epoch: 80. Loss: 0.5914438962936401
Epoch: 90. Loss: 0.5878034234046936
Epoch: 100. Loss: 0.5838867425918579
Epoch: 110. Loss: 0.5791388154029846
Epoch: 120. Loss: 0.5727564692497253
Epoch: 130. Loss: 0.5666334629058838
Epoch: 140. Loss: 0.5580757260322571
Epoch: 150. Loss: 0.5513748526573181
Epoch: 160. Loss: 0.5447183847427368
Epoch: 170. Loss: 0.5398930907249451
Epoch: 180. Loss: 0.5355620384216309
Epoch: 190. Loss: 0.5293852090835571
Epoch: 200. Loss: 0.523815393447876
Epoch: 210. Loss: 0.5191352367401123
Epoch: 220. Loss: 0.514149010181427
Epoch: 230. Loss: 0.5089698433876038
Epoch: 240. Loss: 0.5040460824966431
Epoch: 250. Loss: 0.5015843510627747
Epoch: 260. Loss: 0.49333450198173523
Epoch: 270. Los

### 5. Evaluate the model metrics.

After the model is trained, evaluate the model against the test set.

In [17]:
# Compute the predictions (y_predictions) given the test data
y_predicted = []
with torch.no_grad():
    for i, data in enumerate(X_test):
        predictions = model(data)
        y_predicted.append(predictions.argmax())

# Compare the predicted values for the test set (y_predicted)
# against the expected values (y_test)
print("Classification Report:")
print(classification_report(y_test, y_predicted))

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       107
           1       0.63      0.66      0.65        47

    accuracy                           0.78       154
   macro avg       0.74      0.75      0.74       154
weighted avg       0.78      0.78      0.78       154



The trained model has an accuracy value of 79%.

You can improve the score by retraining the model after more sophisticated data engineering or by tweaking the model's hyper parameters.

### 6. Test the model with sample cases.
Test the model with data from two patients: one patient with diabetes and one patient without diabetes.

In [18]:
# Tuple for textual display of prediction
classes = ('No diabetes', 'Diabetes')


def predict(patients: List[Dict]):
    inputs_dataframe = pd.DataFrame(patients)
    inputs_tensor = torch.FloatTensor(inputs_dataframe.values)
    predictions = []
    for case in inputs_tensor:
        predictions_tensor = model(case)
        prediction_index = predictions_tensor.argmax().item()
        predictions.append(classes[prediction_index])
    return predictions


diabetes_patient = {
    "Pregnancies": 6.0,
    "Glucose": 110.0,
    "BloodPressure": 65.0,
    "SkinThickness": 15.0,
    "Insulin": 1.0,
    "BMI": 45.7,
    "DiabetesPedigreeFunction": 0.627,
    "Age": 50
}

no_diabetes_patient = {
    "Pregnancies": 0,
    "Glucose": 88.0,
    "BloodPressure": 60.0,
    "SkinThickness": 35.0,
    "Insulin": 1.0,
    "BMI": 45.7,
    "DiabetesPedigreeFunction": 0.27,
    "Age": 20
}

predictions = predict([diabetes_patient, no_diabetes_patient])
print(predictions)

['Diabetes', 'No diabetes']
